In [108]:
# NOTEBOOK FOR BATCH PROCESSING SEGMENTATION

In [1]:
import numpy as np
import pandas as pd
import tifffile as tf
from cellpose import models
from cellpose import utils
from skimage.measure import label, regionprops
import os
from tqdm import tqdm
from scipy.spatial.distance import cdist
from skimage.segmentation import expand_labels, watershed
import matplotlib.pyplot as plt

In [2]:
path_unsegmented_datasets='../../figures/2.comparison_between_datasets/input_for_segmentation/'
datasets=os.listdir(path_unsegmented_datasets)

In [3]:
datasets[]

['CosMx',
 'CosMxsegmented_mask.tif',
 'HybrISS',
 'MERFISH_ABI',
 'ResolvedBiosciences',
 'Vizgen',
 'Xenium']

In [3]:
## COSMX DATASET BREAKS TO SEGMENT
## VIZGEN DATASET DOESN'T HAVE A GOOD CORRESPONDANCE TO READS. WE NEED TO APPLY A TRANSFORMATION (MAYBE?)

In [ ]:
for ds in datasets[3:4]:
    print(ds)
    try:
        dapi_image = tf.imread(path_unsegmented_datasets+ds+'/DAPI.tif')
    except:
        dapi_image = tf.imread(path_unsegmented_datasets+ds+'/DAPI.tiff')
    read_positions = pd.read_csv(path_unsegmented_datasets+ds+'/transcripts.csv',index_col=0)
    if ds=='CosMx':
        read_positions['x']=read_positions['y_global_px']
        read_positions['y']=read_positions['x_global_px']
        read_positions['x']=read_positions['x']-np.min(read_positions['x'])
        read_positions['y']=read_positions['y']-np.min(read_positions['y'])
    if ds in ['MERFISH_ABI']:
        read_positions['x']=(read_positions['x_um']-read_positions['x_um'].min())*9.20586
        read_positions['y']=(read_positions['y_um']-read_positions['y_um'].min())*9.20586
    if ds in ['Vizgen']:
        read_positions['x']=read_positions['global_x']*9.20586
        read_positions['y']=read_positions['global_y']*9.20586
    if ds in ['Xenium']:
        read_positions['x']=read_positions['x']*4.70588
        read_positions['y']=read_positions['y']*4.70588
    #plt.figure(figsize=(7,7))
    #plt.imshow(dapi_image,vmax=1200)
    #plt.scatter(read_positions['x'],read_positions['y'],s=0.001,c='red')
   
    # Initialize Cellpose model for nuclei segmentation
    model = models.Cellpose(gpu=False, model_type='nuclei')
    # Segment nuclei
    print('starting segmentation...')
    masks, flows, styles, diams = model.eval(dapi_image, diameter=None, channels=[0, 0])
    # Label the segmented nuclei
    labeled_nuclei = label(masks)
    expanded_nuclei = expand_labels(labeled_nuclei, distance=400)
    from skimage.color import label2rgb
    color1 = label2rgb(expanded_nuclei, bg_label=0)
    plt.imshow(color1)
    centroid_dictx={}
    centroid_dicty={}
    for nucleus_props in regionprops(labeled_nuclei):
        centroid_dictx[nucleus_props.label]=nucleus_props.centroid[0]
        centroid_dicty[nucleus_props.label]=nucleus_props.centroid[1]
    closest_cell=[]
    in_cell=[]
    for ind in tqdm(read_positions.index):
        try:
            in_cell.append(labeled_nuclei[read_positions.loc[ind,'x'],read_positions.loc[ind,'y']])
            closest_cell.append(expanded_nuclei[read_positions.loc[ind,'x'],read_positions.loc[ind,'y']])
        except:
            in_cell.append(0)
            closest_cell.append(0)
    read_positions['in_cell']=in_cell
    read_positions['closest_cell']=closest_cell
    read_positions['closest_cell_x']=read_positions['closest_cell'].map(centroid_dictx)
    read_positions['closest_cell_y']=read_positions['closest_cell'].map(centroid_dicty)
    read_positions['distance_to_centroid']=np.sqrt((read_positions['closest_cell_y']-read_positions['y'])**2 +(read_positions['closest_cell_x']-read_positions['x'])**2)
    cellxgene=pd.crosstab(read_positions['in_cell'],read_positions['gene'])
    cellxgene=cellxgene.loc[~cellxgene.index.isin([0]),:]
    import scanpy as sc
    adata=sc.AnnData(cellxgene)
    adata.obs['x_centroid']=adata.obs.index.astype(int).map(centroid_dictx)
    adata.obs['y_centroid']=adata.obs.index.astype(int).map(centroid_dicty)
    adata.write(path_unsegmented_datasets+ds+'/adata.h5ad')
    read_positions.to_csv(path_unsegmented_datasets+ds+'/transcripts_with_cell_assignment.csv')

ResolvedBiosciences
starting segmentation...


100%|██████████████████████████████████████████████████████████████████████| 8456799/8456799 [05:23<00:00, 26152.23it/s]
/tmp/ipykernel_3438167/1736721645.py:56: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata=sc.AnnData(cellxgene)
/home/sergio/.local/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [ ]:
for ds in datasets[5:6]:
    print(ds)
    try:
        dapi_image = tf.imread(path_unsegmented_datasets+ds+'/DAPI.tif')
    except:
        dapi_image = tf.imread(path_unsegmented_datasets+ds+'/DAPI.tiff')
    read_positions = pd.read_csv(path_unsegmented_datasets+ds+'/transcripts.csv',index_col=0)
    if ds=='CosMx':
        read_positions['x']=read_positions['y_global_px']
        read_positions['y']=read_positions['x_global_px']
        read_positions['x']=read_positions['x']-np.min(read_positions['x'])
        read_positions['y']=read_positions['y']-np.min(read_positions['y'])
    if ds in ['MERFISH_ABI']:
        read_positions['x']=(read_positions['x_um']-read_positions['x_um'].min())*9.20586
        read_positions['y']=(read_positions['y_um']-read_positions['y_um'].min())*9.20586
    if ds in ['Vizgen']:
        read_positions['x']=read_positions['global_x']*9.20586
        read_positions['y']=read_positions['global_y']*9.20586
    if ds in ['Xenium']:
        read_positions['x']=read_positions['x']*4.70588
        read_positions['y']=read_positions['y']*4.70588
    #plt.figure(figsize=(7,7))
    #plt.imshow(dapi_image,vmax=1200)
    #plt.scatter(read_positions['x'],read_positions['y'],s=0.001,c='red')
   
    # Initialize Cellpose model for nuclei segmentation
    model = models.Cellpose(gpu=False, model_type='nuclei')
    # Segment nuclei
    print('starting segmentation...')
    masks, flows, styles, diams = model.eval(dapi_image, diameter=None, channels=[0, 0])
    # Label the segmented nuclei
    labeled_nuclei = label(masks)
    expanded_nuclei = expand_labels(labeled_nuclei, distance=400)
    from skimage.color import label2rgb
    color1 = label2rgb(expanded_nuclei, bg_label=0)
    plt.imshow(color1)
    centroid_dictx={}
    centroid_dicty={}
    for nucleus_props in regionprops(labeled_nuclei):
        centroid_dictx[nucleus_props.label]=nucleus_props.centroid[0]
        centroid_dicty[nucleus_props.label]=nucleus_props.centroid[1]
    closest_cell=[]
    in_cell=[]
    for ind in tqdm(read_positions.index):
        try:
            in_cell.append(labeled_nuclei[read_positions.loc[ind,'x'],read_positions.loc[ind,'y']])
            closest_cell.append(expanded_nuclei[read_positions.loc[ind,'x'],read_positions.loc[ind,'y']])
        except:
            in_cell.append(0)
            closest_cell.append(0)
    read_positions['in_cell']=in_cell
    read_positions['closest_cell']=closest_cell
    read_positions['closest_cell_x']=read_positions['closest_cell'].map(centroid_dictx)
    read_positions['closest_cell_y']=read_positions['closest_cell'].map(centroid_dicty)
    read_positions['distance_to_centroid']=np.sqrt((read_positions['closest_cell_y']-read_positions['y'])**2 +(read_positions['closest_cell_x']-read_positions['x'])**2)
    cellxgene=pd.crosstab(read_positions['in_cell'],read_positions['gene'])
    cellxgene=cellxgene.loc[~cellxgene.index.isin([0]),:]
    import scanpy as sc
    adata=sc.AnnData(cellxgene)
    adata.obs['x_centroid']=adata.obs.index.astype(int).map(centroid_dictx)
    adata.obs['y_centroid']=adata.obs.index.astype(int).map(centroid_dicty)
    adata.write(path_unsegmented_datasets+ds+'/adata.h5ad')
    read_positions.to_csv(path_unsegmented_datasets+ds+'/transcripts_with_cell_assignment.csv')

Xenium
starting segmentation...


In [4]:
ds='Xenium'

In [6]:
dapi_image = tf.imread(path_unsegmented_datasets+ds+'/DAPI.tiff')
tile_size=[2000,2000]
overlap=100
tile_positions = []
segmented_masks = []
from tqdm import tqdm

In [26]:
read_positions = pd.read_csv(path_unsegmented_datasets+ds+'/transcripts.csv',index_col=0)

In [28]:
read_positions['x']=read_positions['y_global_px']
read_positions['y']=read_positions['x_global_px']
read_positions['x']=read_positions['x']-np.min(read_positions['x'])
read_positions['y']=read_positions['y']-np.min(read_positions['y'])

In [ ]:
height, width = dapi_image.shape[:2]

for y in tqdm(range(0, height, tile_size[1] - overlap)):
    for x in tqdm(range(0, width, tile_size[0] - overlap)):
        # Calculate tile boundaries
        x1, x2 = x, min(x + tile_size[0], width)
        y1, y2 = y, min(y + tile_size[1], height)

        # Extract the tile
        tile = dapi_image[y1:y2, x1:x2]

        # Store the tile position
        tile_positions.append((x1, y1))

        # Step 2: Segment each tile with Cellpose
        # Initialize Cellpose model
        model = models.Cellpose(gpu=False)  # You can set GPU to True if available

        # Run Cellpose on each tile
        masks, _, _, _ = model.eval(tile, channels=[0, 0], diameter=None)  # Modify channels and diameter as needed
        
        segmented_masks.append(masks)

  0%|                                                                                            | 0/26 [00:00<?, ?it/s]

In [ ]:
# Step 3: Recompose the segmented masks into a single mask
full_mask = np.zeros((height, width), dtype=np.uint8)

for mask, position in tqdm(zip(segmented_masks, tile_positions)):
    x, y = position
    h, w = mask.shape[:2]
    full_mask[y:y + h, x:x + w] = mask # Assuming you want only one channel of the mask

In [ ]:
np.max(full_mask)

In [ ]:
# Optional: Save the full mask to a file
tf.imwrite(path_unsegmented_datasets+ds+"/segmented_mask.tif", full_mask)

In [29]:
read_positions

,x_global_px,y_global_px,target,x,y
0,-494466.803333,9108.900,Gfap,39522.820,3903.496667
1,-494237.833333,10084.600,Ngf,40498.520,4132.466667
2,-494239.533333,10080.917,Igfbp7,40494.837,4130.766667
3,-494227.493333,10072.143,Gnaq,40486.063,4142.806667
4,-494219.603333,10065.550,Cldn5,40479.470,4150.696667
...,...,...,...,...,...
116159822,-454293.000000,-22302.590,Apoe,8111.330,44077.300000
116159823,-454296.300000,-22325.660,Prkacb,8088.260,44074.000000
116159824,-454262.000000,-22325.610,Cox6c,8088.310,44108.300000
116159825,-454297.800000,-22326.310,Malat1,8087.610,44072.500000


In [ ]:
# Display the final mask (for visualization purposes)
plt.figure(figsize=(10,10))
plt.imshow(full_mask,vmax=1)
plt.scatter(read_positions.x,read_positions.y,s=0.01)

In [ ]:
labeled_nuclei = label(masks)
expanded_nuclei = expand_labels(labeled_nuclei, distance=400)
from skimage.color import label2rgb
color1 = label2rgb(expanded_nuclei, bg_label=0)
plt.imshow(color1)
centroid_dictx={}
centroid_dicty={}
for nucleus_props in regionprops(labeled_nuclei):
    centroid_dictx[nucleus_props.label]=nucleus_props.centroid[0]
    centroid_dicty[nucleus_props.label]=nucleus_props.centroid[1]
closest_cell=[]
in_cell=[]
for ind in tqdm(read_positions.index):
    try:
        in_cell.append(labeled_nuclei[read_positions.loc[ind,'x'],read_positions.loc[ind,'y']])
        closest_cell.append(expanded_nuclei[read_positions.loc[ind,'x'],read_positions.loc[ind,'y']])
    except:
        in_cell.append(0)
        closest_cell.append(0)
read_positions['in_cell']=in_cell
read_positions['closest_cell']=closest_cell
read_positions['closest_cell_x']=read_positions['closest_cell'].map(centroid_dictx)
read_positions['closest_cell_y']=read_positions['closest_cell'].map(centroid_dicty)
read_positions['distance_to_centroid']=np.sqrt((read_positions['closest_cell_y']-read_positions['y'])**2 +(read_positions['closest_cell_x']-read_positions['x'])**2)
cellxgene=pd.crosstab(read_positions['in_cell'],read_positions['gene'])
cellxgene=cellxgene.loc[~cellxgene.index.isin([0]),:]
import scanpy as sc
adata=sc.AnnData(cellxgene)
adata.obs['x_centroid']=adata.obs.index.astype(int).map(centroid_dictx)
adata.obs['y_centroid']=adata.obs.index.astype(int).map(centroid_dicty)
adata.write(path_unsegmented_datasets+ds+'/adata.h5ad')
read_positions.to_csv(path_unsegmented_datasets+ds+'/transcripts_with_cell_assignment.csv')

In [ ]:


# Step 2: Segment each tile with Cellpose
segmented_masks = []

# Initialize Cellpose model
model = models.Cellpose(gpu=False)  # You can set GPU to True if available

for tile in tiles:
    # Run Cellpose on each tile
    masks, _, _, _ = model.eval(tile, channels=[0, 0], diameter=30)  # Modify channels and diameter as needed
    segmented_masks.append(masks)

# Step 3: Recompose the segmented masks into a single mask
original_image_shape = large_image.shape[:2]

full_mask = np.zeros(original_image_shape, dtype=np.uint8)

for mask, position in zip(segmented_masks, tile_positions):
    x, y = position
    h, w = mask.shape[:2]
    full_mask[y:y + h, x:x + w] = mask[..., 0]  # Assuming you want only one channel of the mask

# 'full_mask' now contains the segmentation mask for the entire image

# Optional: Save the full mask to a file
cv2.imwrite("segmented_mask.jpg", full_mask)

# Optional: Perform post-processing on the 'full_mask' if needed

# Display the final mask (for visualization purposes)
cv2.imshow("Segmented Mask", full_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import numpy as np

# Assuming 'segmented_masks' is a list of segmented masks for each tile
# and 'tile_positions' is a list of the positions of each tile in the original image

# Determine the shape of the original image
original_image_shape = (height, width)

# Create an empty mask of the same shape as the original image
full_mask = np.zeros(original_image_shape, dtype=np.uint8)

# Iterate through the segmented masks and tile positions
for mask, position in zip(segmented_masks, tile_positions):
    x, y = position  # Position of the top-left corner of the tile
    h, w = mask.shape  # Height and width of the tile
    full_mask[y:y + h, x:x + w] = mask

# 'full_mask' now contains the segmentation mask for the entire image


In [8]:
datasets=['CosMx','Vizgen','Xenium']

In [6]:
for ds in datasets:
    if os.path.exists(path_unsegmented_datasets+ds+'/transcripts_with_cell_assignment.csv'):
        print(ds)
   

HybrISS
MERFISH_ABI
ResolvedBiosciences
